# Get Uniprot IDS for influenza
## Christian Roncal Cmsc499a Dr. Leiserson

In [1]:
import os 
import pandas as pd
import numpy as np
from utils import *
import re

In [2]:
os.listdir()

['make_matrix.py',
 'utils.py',
 'dataprocessing_playground.ipynb',
 'influenza',
 'flustrains.txt',
 '.ipynb_checkpoints',
 'influenza_a_processing.ipynb',
 '__pycache__']

# 1 load data

In [3]:
flu_csv = './influenza/flu_phi.csv'

In [13]:
fludf = pd.read_csv(flu_csv, index_col=0)

In [14]:
fludf.head()

,Pathogen,Taxonomy ID,VirusUniprot,Pathogen Protein,HumanUniprot,Human Protein,Experimental Method,Pubmed ID
0,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03433,PA_I34A1,P49736,MCM2_HUMAN,anti bait coimmunoprecipitation,17932485
1,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03466,NCAP_I34A1,P49736,MCM2_HUMAN,anti bait coimmunoprecipitation,17932485
2,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03428,PB2_I34A1,P49736,MCM2_HUMAN,anti bait coimmunoprecipitation,17932485
3,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03485,M1_I34A1,P15311,EZRI_HUMAN,anti bait coimmunoprecipitation,17022977
4,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03485,M1_I34A1,P11142,HSP7C_HUMAN,anti bait coimmunoprecipitation,17022977


In [15]:
len(fludf['Pathogen'].unique()) #not all of this will be of interest

526

# 2 Extract relevant strains

Use regex to get strains we're interested in: H1N1|H3N2|H5N1|H7N3

In [16]:
regex = 'H1N1|H3N2|H5N1|H7N3'
fludf = regex_select(fludf, 'Pathogen', regex)

Pathogen H1N1|H3N2|H5N1|H7N3


In [17]:
fludf.head()

,Pathogen,Taxonomy ID,VirusUniprot,Pathogen Protein,HumanUniprot,Human Protein,Experimental Method,Pubmed ID
0,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03433,PA_I34A1,P49736,MCM2_HUMAN,anti bait coimmunoprecipitation,17932485
1,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03466,NCAP_I34A1,P49736,MCM2_HUMAN,anti bait coimmunoprecipitation,17932485
2,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03428,PB2_I34A1,P49736,MCM2_HUMAN,anti bait coimmunoprecipitation,17932485
3,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03485,M1_I34A1,P15311,EZRI_HUMAN,anti bait coimmunoprecipitation,17022977
4,Influenza A virus STRAIN A / PUERTO RICO / 8 /...,211044,P03485,M1_I34A1,P11142,HSP7C_HUMAN,anti bait coimmunoprecipitation,17022977


In [18]:
len(fludf['Pathogen'].unique())

52

# 3 Get virus|human uniprots of interactions

get a df with cols virus uniprot and human uniprot for matrix interaction processing.

In [19]:
# drop cols we don't need
fludf = fludf[['VirusUniprot', 'HumanUniprot']]